In [1]:
from semopy import Model
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from semopy import Optimizer
from semopy.model_generator import generate_model
from semopy.inspector import inspect
from semopy.stats import calculate_p_values, calc_gfi, gather_statistics

In [2]:
data = pd.read_csv("file.csv")
data.head()

,y1,y2,x1,x2,x3,x4,x5,x6
0,5,5,5,5,5,5,5,5
1,5,5,5,5,5,5,5,5
2,4,4,4,4,4,4,4,4
3,4,4,4,4,4,4,4,4
4,1,1,2,2,2,2,2,1


In [3]:
mod = """ # Measurement Model
          eta1 =~ x1 + x2 + x3
          eta2 =~ x4 + x5 + x6
          eta =~ y1 + y2
          
          # Structural Model
          eta ~ eta1 + eta2
          
          # Residual Correlations
          x1 ~~ x4
          x2 ~~ x5
          x3 ~~ x6
      """
model = Model(mod)
model.load_dataset(data)

In [4]:
opt_mlw = Optimizer(model)
opt_uls = Optimizer(model)
opt_gls = Optimizer(model)

# And now, we run the optimisation sequences.
lf_mlw = opt_mlw.optimize(objective='MLW') # Although MLW is default, we still provide it here for clarity.
lf_uls = opt_uls.optimize(objective='ULS')
lf_gls = opt_gls.optimize(objective='GLS')

print('Resultant objective functions'' values are:')
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}'.format(lf_mlw, lf_uls, lf_gls))

Resultant objective functions values are:
MLW: 1.275, ULS: 0.192, GLS: 1.806


In [5]:
opt_uls_adam = Optimizer(model)
lf_uls_adam = opt_uls_adam.optimize(objective='ULS', method='Adam', chunk_size=100, num_epochs=2000)
print('ULS after Adam: {:.3f}'.format(lf_uls_adam))

ULS after Adam: 0.192


In [6]:
lf_uls_adam_slsqp = opt_uls_adam.optimize(method='SLSQP')
print('ULS after Adam after SLSQP: {:.3f}'.format(lf_uls_adam_slsqp))

ULS after Adam after SLSQP: 1.275


In [7]:
inspect(opt_uls)

,lval,op,rval,Value,SE,Z-score,P-value
6,eta,=~,y2,0.826376,0.128337,6.439087,1.201943e-10
2,eta1,=~,x2,1.347495,0.293036,4.598399,4.257502e-06
3,eta1,=~,x3,1.519331,0.307714,4.937478,7.913944e-07
4,eta2,=~,x5,1.388807,0.724993,1.915615,5.541410e-02
5,eta2,=~,x6,1.581242,0.895972,1.764833,7.759186e-02
0,eta,~,eta1,2.154062,1.051181,2.049183,4.044426e-02
1,eta,~,eta2,-0.753287,1.234108,-0.610389,5.416039e-01
7,eta,~~,eta,0.134991,0.148400,0.909643,3.630108e-01
8,eta1,~~,eta1,0.325119,0.141303,2.300867,2.139913e-02
9,eta1,~~,eta2,0.293968,0.191076,1.538491,1.239285e-01


In [8]:
print(inspect(opt_uls, mode='mx'))

Beta:
      eta      eta1      eta2
eta   0.0  2.154062 -0.753287
eta1  0.0  0.000000  0.000000
eta2  0.0  0.000000  0.000000
Lambda:
         eta      eta1      eta2
x1  0.000000  1.000000  0.000000
x2  0.000000  1.347495  0.000000
x3  0.000000  1.519331  0.000000
x4  0.000000  0.000000  1.000000
x5  0.000000  0.000000  1.388807
x6  0.000000  0.000000  1.581242
y1  1.000000  0.000000  0.000000
y2  0.826376  0.000000  0.000000
Psi:
           eta      eta1      eta2
eta   0.134991  0.000000  0.000000
eta1  0.000000  0.325119  0.293968
eta2  0.000000  0.293968  0.219979
Theta:
          x1        x2        x3        x4        x5        x6        y1  \
x1  0.181504  0.000000  0.000000  0.235331  0.000000  0.000000  0.000000   
x2  0.000000  0.256551  0.000000  0.000000 -0.015165  0.000000  0.000000   
x3  0.000000  0.000000  0.243837  0.000000  0.000000 -0.057843  0.000000   
x4  0.235331  0.000000  0.000000  1.689287  0.000000  0.000000  0.000000   
x5  0.000000 -0.015165  0.000000  0.0

In [9]:
s = gather_statistics(opt_uls)
print(s)

SEMStatistics(dof=14.0, ml=-184.278031738658, fit_val=0.19176380950153094, chi2=(4.410567618535212, 0.9924460389989741), dof_baseline=28.0, chi2_baseline=384.2416372154187, rmsea=0, cfi=1.0269183368244685, gfi=0.9885213699106157, agfi=0.9770427398212314, nfi=0.9885213699106157, tli=1.0538366736489373, aic=412.556063477316, bic=437.5369362277573, params=[ParametersStatistics(value=2.1540622219691907, se=1.0511811636430433, zscore=2.049182668477316, pvalue=0.04044425616840508), ParametersStatistics(value=-0.7532866496693092, se=1.2341083393264425, zscore=-0.6103894007229796, pvalue=0.5416038877343083), ParametersStatistics(value=1.347495347915129, se=0.29303576700483924, zscore=4.59839889745704, pvalue=4.257502356441734e-06), ParametersStatistics(value=1.5193311987681783, se=0.30771403563937344, zscore=4.937477731902443, pvalue=7.913943962378767e-07), ParametersStatistics(value=1.3888066446160297, se=0.7249925160964029, zscore=1.9156151460595812, pvalue=0.055414100243476705), ParametersS

In [12]:
print('MLW: {:.3f}, ULS: {:.3f}, GLS: {:.3f}, GLS after Adam : {:.3f}'.format(calc_gfi(opt_mlw),
                                                                                         calc_gfi(opt_uls),
                                                                                         calc_gfi(opt_gls),
                                                                                         calc_gfi(opt_uls_adam)))

MLW: 0.832, ULS: 0.989, GLS: 0.531, GLS after Adam : 0.832
